<a href="https://colab.research.google.com/github/SuryakalaR/AI-knowledge-graph-Builder-for-enterprise-intelligence/blob/module_3/Milestone_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install langchain langchain-community langchain-openai faiss-cpu neo4j pandas

In [ ]:
pip install requests==2.32.4

In [ ]:
import requests
print(requests.__version__)

In [ ]:
!pip install requests==2.32.5

In [ ]:
import requests
print(requests.__version__)


In [ ]:
!pip install -U langchain langchain-community faiss-cpu sentence-transformers


In [ ]:
from langchain_community.vectorstores import FAISS
print("FAISS + LangChain ready ✅")

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd

filename = list(uploaded.keys())[0]
df = pd.read_csv(filename)

print("Dataset loaded ✅")
df.head()

In [ ]:
import re

def extract_entities(text):
    persons = re.findall(r"\b[A-Z][a-z]+ [A-Z][a-z]+\b", text)
    organizations = re.findall(r"\b(Google|Microsoft|Amazon|OpenAI)\b", text)
    return persons, organizations

df["text"] = df.iloc[:, 0].astype(str)

entity_records = []

for text in df["text"]:
    persons, orgs = extract_entities(text)
    entity_records.append({
        "text": text,
        "persons": persons,
        "organizations": orgs
    })

entity_records[:2]

In [ ]:
from langchain_core.documents import Document

In [ ]:
from langchain_core.documents import Document

documents = []

for item in entity_records:
    documents.append(
        Document(
            page_content=item["text"],
            metadata={
                "persons": item["persons"],
                "organizations": item["organizations"]
            }
        )
    )

len(documents)


In [ ]:
import pandas as pd
import re
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


In [ ]:
SAMPLE_SIZE = 1000   # safe size for Colab

sampled_documents = documents[:SAMPLE_SIZE]

print("Sample size:", len(sampled_documents))


In [ ]:
MAX_CHARS = 300

for d in sampled_documents:
    d.page_content = d.page_content[:MAX_CHARS]

In [ ]:
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(sampled_documents, embeddings)

print("FAISS vector store created successfully ✅")

In [ ]:
query = "Who is connected to Google?"

results = vectorstore.similarity_search(query, k=3)

for r in results:
    print(r.page_content)
    print(r.metadata)
    print("-" * 40)

In [ ]:
# Semantic search query
query = "Who is connected to Google?"

results = vectorstore.similarity_search(query, k=3)

print("Semantic search results:\n")

for i, r in enumerate(results, 1):
    print(f"Result {i}")
    print("Text:", r.page_content)
    print("Metadata:", r.metadata)
    print("-" * 40)

In [ ]:
triples = []

for r in results:
    persons = r.metadata.get("persons", [])
    orgs = r.metadata.get("organizations", [])

    for p in persons:
        for o in orgs:
            triples.append((p, "SEMANTICALLY_RELATED", o))

triples


In [ ]:
!pip install neo4j


In [ ]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver(
    "bolt://127.0.0.1:7687",   # or Aura URI
    auth=("module_3", "123456789")
)

with driver.session() as session:
    for s, r, o in triples:
        session.run("""
        MERGE (p:Person {name:$s})
        MERGE (o:Organization {name:$o})
        MERGE (p)-[:SEMANTIC_RESULT]->(o)
        """, s=s, o=o)

print("Semantic search results stored in Neo4j ✅")
